<a href="https://colab.research.google.com/github/pratiksrm99/Abstractive_Text_Summerization/blob/main/Abstractive_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets
!pip install pywsd
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 5.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wn: filename=wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from datasets import load_dataset
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from pywsd.lesk import adapted_lesk, simple_lesk, cosine_lesk
import pandas as pd
import numpy as np
import spacy
import re
from gensim.models import Word2Vec

Warming up PyWSD (takes ~10 secs)... took 10.463255167007446 secs.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
dataset = load_dataset('gigaword')

Generating train split:   0%|          | 0/3803957 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/189651 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1951 [00:00<?, ? examples/s]

Dataset gigaword downloaded and prepared to /root/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [36]:
df = pd.DataFrame(columns= ["Document", "Summary"])
df["Document"] = dataset.get("train")["document"]
df["Summary"] = dataset.get("train")["summary"]
df = df.loc[0:9999,:]
df["Document"] = df["Document"].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))
df["Document"] = df["Document"].apply(lambda x: re.sub(r'(?:^| )\w(?:$| )', ' ', x).strip())
df

,Document,Summary
0,australia current account deficit shrunk by re...,australian current account deficit narrows sha...
1,at least two people were killed in suspected b...,at least two dead in southern philippines blast
2,australian shares closed down percent monday f...,australian stocks close down #.# percent
3,south korea nuclear envoy kim sook urged north...,envoy urges north korea to restart nuclear dis...
4,south korea on monday announced sweeping tax r...,skorea announces tax cuts to stimulate economy
...,...,...
9995,sri lanka is reviewing an agreement to allow t...,sri lanka to review agreement for setting up v...
9996,the UNK liner qe finally left the port of new ...,qe# leaves new york
9997,zimbabwe only independent daily newspaper the ...,zimbabwe 's only independent daily winds up
9998,cambodia foreign minister saturday ordered an ...,cambodian government investigates missing germ...


In [6]:
# df["Split_text"] = df["Document"].apply(str.split)
# df["Article_length"] = df["Split_text"].apply(len)

In [31]:
df.head()

,Document,Summary
0,australia current account deficit shrunk by re...,australian current account deficit narrows sha...
1,at least two people were killed in suspected b...,at least two dead in southern philippines blast
2,australian shares closed down percent monday f...,australian stocks close down #.# percent
3,south korea nuclear envoy kim sook urged north...,envoy urges north korea to restart nuclear dis...
4,south korea on monday announced sweeping tax r...,skorea announces tax cuts to stimulate economy


# **Implementation of Algorithm 1 from the paper**

Algorithm 1 Level-driven text generalization (LG)

Require: text, wsdText, F, T, θd
, θf

1: genText ← text

2: for all token ∈ text do

3: ftoken ← Frequency of token from F

4: if ftoken ≤ θf
then

5: c ← WSD of token form wsdText

6: Pc ← Path of hypernyms of c from T

7: dc ← taxonomy depth of concept c

8: fc ← Frequency of c from F

9: while fc < θf and dc > θd do

10: c ← hypernym of c from Ptoken

11: dc ← taxonomy depth of c

12: fc ← Frequency of c from F

13: end while

14: if (word of c) 6= token then

15: genText ← generalize token of genText to c

16: F ← (F \ {(token, ftoken)}) ∪ {(token, ftoken − 1)}

17: F ← (F \ {(c, fc
)}) ∪ {(c, fc + 1)}

18: end if

19: end if

20: end for

21: return genText

In [32]:
sent = "he is sitting on the bank of the river watching a speedboat in japan."
nlp = spacy.load("en_core_web_sm")

def one_func(x):
  try:
    if x.pos_=="NOUN":
      return adapted_lesk(sent,x.text).name()
    else:
      return x.text
  except:
    return x.text

def wsd_text(sent):
  doc = nlp(sent)
  test_df = pd.DataFrame(doc,columns=["text"])
  test_df["wsd"] = test_df["text"].apply(one_func)
  return " ".join(test_df["wsd"])

In [37]:
df["WSD Text"] = df["Document"].apply(wsd_text)

In [38]:
df.head()

,Document,Summary,WSD Text
0,australia current account deficit shrunk by re...,australian current account deficit narrows sha...,australia current score.n.05 deficit.n.04 shru...
1,at least two people were killed in suspected b...,at least two dead in southern philippines blast,at least two people.v.02 were killed in suspec...
2,australian shares closed down percent monday f...,australian stocks close down #.# percent,australian share.v.05 closed down percentage.n...
3,south korea nuclear envoy kim sook urged north...,envoy urges north korea to restart nuclear dis...,south korea nuclear envoy.n.03 kim sook urged ...
4,south korea on monday announced sweeping tax r...,skorea announces tax cuts to stimulate economy,south korea on monday announced sweeping tax.v...


In [39]:
a = df['WSD Text'].str.cat(sep=' ')
test_list = re.findall("[\w]+\..{1}\..{2}",a)
word_dist = FreqDist(test_list)
word_dist

FreqDist({'dollar.n.04': 1059, 'percentage.n.01': 945, 'politics.n.02': 883, 'official.s.05': 881, 'year.n.03': 834, 'state.n.04': 793, 'share.v.05': 775, 'marketplace.n.02': 771, 'sidereal_day.n.01': 627, 'agency.n.01': 626, ...})

In [40]:
thetha_d = 5
thetha_f = 100

def to_generalize(sent,thetha_d,thetha_f,word_dist):

  try:
    word_lst = [word for word in re.findall('[\w]+\..{1}\..{2}', sent) \
                if (word_dist[word] < thetha_f) and (len(wn.synset(word).hypernym_paths()[0])-1 > thetha_d)]

    for word in word_lst:
      while word_dist[word] < thetha_f and (len(wn.synset(word).hypernym_paths()[0])-1 > thetha_d):
        try:
          sent = sent.replace(word, str(wn.synset(word).hypernyms()[0].name()))
        except:
          sent = sent.replace(i, str(wn.synset(word).attributes()[0].name()))
        finally:
          word = wn.synset(word).hypernyms()[0].name()
  except:
    return "Error while generalizing"

  return sent

In [41]:
df["Generalized Text"] = df["WSD Text"].apply(lambda x: to_generalize(x,thetha_d,thetha_f,word_dist))
# xyz=23
# print(df["WSD Text"][xyz],"\n",df["Document"][xyz])

In [42]:
df[["Document","WSD Text", "Generalized Text"]]

,Document,WSD Text,Generalized Text
0,australia current account deficit shrunk by re...,australia current score.n.05 deficit.n.04 shru...,australia current reason.n.01 deficit.n.04 shr...
1,at least two people were killed in suspected b...,at least two people.v.02 were killed in suspec...,at least two people.v.02 were killed in suspec...
2,australian shares closed down percent monday f...,australian share.v.05 closed down percentage.n...,australian share.v.05 closed down percentage.n...
3,south korea nuclear envoy kim sook urged north...,south korea nuclear envoy.n.03 kim sook urged ...,south korea nuclear matter.n.06 kim sook urged...
4,south korea on monday announced sweeping tax r...,south korea on monday announced sweeping tax.v...,south korea on monday announced sweeping tax.v...
...,...,...,...
9995,sri lanka is reviewing an agreement to allow t...,sri lanka is reviewing an agreement.n.06 to al...,Error while generalizing
9996,the UNK liner qe finally left the port of new ...,the UNK lining.n.01 qe finally left the port.v...,the UNK covering.n.02 qe finally left the port...
9997,zimbabwe only independent daily newspaper the ...,zimbabwe only independent daily newspaper.n.04...,zimbabwe only independent daily paper.n.01 the...
9998,cambodia foreign minister saturday ordered an ...,cambodia foreign minister saturday ordered an ...,cambodia foreign minister saturday ordered an ...


In [43]:
test_gen=to_generalize(wsd_text(sent),thetha_d,thetha_f,word_dist)
print(sent,"\n",test_gen)
wn.synset('instrumentality.n.03').definition()
new_df = df[df["Generalized Text"]!="Error while generalizing"]
new_df = new_df[["Generalized Text", "Summary"]]

he is sitting on the bank of the river watching a speedboat in japan. 
 he is sitting on the bank.n.01 of the river.n.01 watching a instrumentality.n.03 in japan .


In [44]:
new_df["Gen_text_list"] = new_df["Generalized Text"].apply(str.split)
i=4
sentences = new_df["Gen_text_list"].to_list()

In [45]:
w2v = Word2Vec(sentences, min_count=1, size = 300, window=5,alpha=0.025, min_alpha=0.001, iter=10,)

In [52]:
w2v.wv.get_vector('grim')

array([ 2.90648676e-02, -6.50759861e-02,  8.40820372e-03,  9.80888680e-03,
        1.61662586e-02, -4.63743582e-02, -3.57232243e-02,  1.38300359e-02,
       -6.51739398e-03,  2.91736051e-03, -2.12171134e-02, -5.26064169e-03,
       -1.27106132e-02, -1.49399675e-02,  1.13510583e-02, -1.60694066e-02,
       -6.77530421e-03,  2.79308148e-02,  2.32219882e-02, -3.05616367e-03,
       -6.04444509e-03, -3.77940238e-02, -5.40441554e-03,  4.93479194e-03,
       -1.40620896e-03, -2.56848205e-02, -1.88036524e-02,  2.64354758e-02,
       -1.66601855e-02,  2.30214372e-02,  2.74383612e-02,  3.60056460e-02,
       -1.91083513e-02,  3.30590497e-04,  1.37759615e-02, -2.86490861e-02,
       -1.64769460e-02, -8.73791613e-03, -2.19178926e-02, -4.60858282e-05,
       -1.26093999e-02, -1.13146165e-02, -1.83718707e-02,  1.41373952e-04,
        2.79122684e-02,  1.46115366e-02,  1.13719711e-02,  6.87299948e-03,
       -1.96748786e-02, -1.89122874e-02, -8.30441248e-03, -5.98773267e-03,
       -2.23380942e-02, -